In [7]:
pip install nutter


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114262 sha256=57877106d4e0e60eb5b2eb63d6c27a7bf3813932f8d4b5cc40203129c8ee3d5f
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\9e\5b\45\29f72e55d87a29426b04b3cfdf20325c079eb97ab74f59017d
Successfully built fire
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install scipy


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install opencv-python

In [2]:
pip install librosa


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install soundfile

In [5]:
pip install opencv-contrib-python


   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/46.2 MB 1.7 MB/s eta 0:00:28
    --------------------------------------- 0.8/46.2 MB 1.7 MB/s eta 0:00:28
   - -------------------------------------- 1.3/46.2 MB 1.6 MB/s eta 0:00:29
   - -------------------------------------- 1.6/46.2 MB 1.6 MB/s eta 0:00:28
   - -------------------------------------- 2.1/46.2 MB 1.8 MB/s eta 0:00:26
   -- ------------------------------------- 2.4/46.2 MB 1.7 MB/s eta 0:00:26
   -- ------------------------------------- 2.9/46.2 MB 1.8 MB/s eta 0:00:25
   -- ------------------------------------- 3.4/46.2 MB 1.8 MB/s eta 0:00:24
   --- ------------------------------------ 3.9/46.2 MB 1.9 MB/s eta 0:00:22
   ---- ----------------------------------- 4.7/46.2 MB 2.1 MB/s eta 0:00:20
   ---- ----------------------------------- 5.2/46.2 MB 2.2 MB/s eta 0:00:19
   ----- ----

PRIMEIRO CODIGO

In [6]:
import cv2
import numpy as np
import soundfile as sf
import os

# Carregar imagem de ultrassonografia
def load_ultrasound(image_path):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"O arquivo {image_path} não foi encontrado!")
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Erro ao carregar a imagem! Verifique o formato.")
    img = cv2.resize(img, (128, 128))  # Redimensionar para 128x128
    img = img / 255.0  # Normalizar os valores de pixel entre 0 e 1
    return img

# Converter imagem para som simulando batidas cardíacas
def ultrasound_to_heartbeat(image):
    heartbeat = []
    for row in image:
        intensity = np.mean(row)  # Média da intensidade de cada linha
        frequency = np.interp(intensity, [0, 1], [60, 160])  # Mapear intensidade para BPM
        print(f"Intensidade: {intensity}, Frequência: {frequency}")  # Debug
        duration = 0.2  # Duração de cada batida
        t = np.linspace(0, duration, int(44100 * duration), endpoint=False)
        beat = 0.5 * np.sin(2 * np.pi * frequency * t)  # Gerar som de batida
        heartbeat.extend(beat)
    return np.array(heartbeat)

# Salvar som em um arquivo WAV
def save_heartbeat(sound, file_path):
    if sound.size == 0:
        raise ValueError("O som gerado está vazio!")
    sound = sound / np.max(np.abs(sound))  # Normalizar o som para o intervalo [-1, 1]
    sf.write(file_path, sound, samplerate=44100)

# Caminho da imagem de ultrassom
image_path = 'frequencia_bebe.jpg'
try:
    image = load_ultrasound(image_path)

    # Gerar som do coração
    heartbeat_sound = ultrasound_to_heartbeat(image)
    print(f"Tamanho do som gerado: {heartbeat_sound.shape}")

    # Salvar o som
    save_heartbeat(heartbeat_sound, 'C:/Users/Admin/Documents/baby.wav')
    print("Arquivo de som gerado: 'baby.wav'")
except Exception as e:
    print(f"Erro: {e}")



Intensidade: 0.27052696078431376, Frequência: 87.05269607843138
Intensidade: 0.19304534313725488, Frequência: 79.30453431372548
Intensidade: 0.1834252450980392, Frequência: 78.34252450980392
Intensidade: 0.1734987745098039, Frequência: 77.34987745098039
Intensidade: 0.1590073529411765, Frequência: 75.90073529411765
Intensidade: 0.14301470588235293, Frequência: 74.30147058823529
Intensidade: 0.1400735294117647, Frequência: 74.00735294117646
Intensidade: 0.15906862745098038, Frequência: 75.90686274509804
Intensidade: 0.17840073529411765, Frequência: 77.84007352941177
Intensidade: 0.20796568627450981, Frequência: 80.79656862745098
Intensidade: 0.2389093137254902, Frequência: 83.89093137254902
Intensidade: 0.22365196078431374, Frequência: 82.36519607843138
Intensidade: 0.19549632352941176, Frequência: 79.54963235294117
Intensidade: 0.20398284313725493, Frequência: 80.3982843137255
Intensidade: 0.19666053921568627, Frequência: 79.66605392156862
Intensidade: 0.20052083333333334, Frequência: 

CODIGO COM FILTRO

In [14]:
import cv2
import numpy as np
import soundfile as sf
from scipy.signal import butter, lfilter

# Função para carregar imagem de ultrassonografia
def load_ultrasound(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))  # Redimensionar para 128x128
    img = img / 255.0  # Normalizar os valores de pixel entre 0 e 1
    return img

# Função para gerar som de batidas cardíacas a partir da imagem
def ultrasound_to_heartbeat(image):
    heartbeat = []
    for row in image:
        intensity = np.mean(row)  # Média da intensidade de cada linha
        frequency = np.interp(intensity, [0, 1], [60, 160])  # Mapear intensidade para BPM
        duration = 0.08 # Duração de cada batida
        t = np.linspace(0, duration, int(44100 * duration), endpoint=False)
        beat = 0.5 * np.sin(2 * np.pi * frequency * t)  # Gerar som de batida
        heartbeat.extend(beat)
    return np.array(heartbeat)

# Função para adicionar pausas entre batidas cardíacas
def add_pauses(sound, pause_duration=0.5, fs=44100):
    pause = np.zeros(int(fs * pause_duration))  # Criar intervalo de silêncio
    return np.concatenate([sound, pause])

# Função para aplicar filtro passa-baixa
def lowpass_filter(data, cutoff=150, fs=44100, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    return y

# Função para salvar som em um arquivo WAV
def save_heartbeat(sound, file_path):
    sound = sound / np.max(np.abs(sound))  # Normalizar o som para o intervalo [-1, 1]
    sf.write(file_path, sound, samplerate=44100)

# Caminho da imagem de ultrassom
image_path = 'frequencia_bebe.jpg'

# Processamento da imagem
image = load_ultrasound(image_path)

# Gerar som do coração
heartbeat_sound = ultrasound_to_heartbeat(image)

# Adicionar pausas entre batidas
heartbeat_with_pauses = []
fs = 44100  # Taxa de amostragem
for i in range(0, len(heartbeat_sound), int(fs * 0.2)):
    heartbeat_with_pauses.extend(add_pauses(heartbeat_sound[i:i + int(fs * 0.2)]))
heartbeat_sound = np.array(heartbeat_with_pauses)

# Aplicar filtro passa-baixa para suavizar o som
heartbeat_sound = lowpass_filter(heartbeat_sound)

# Salvar o som filtrado
save_heartbeat(heartbeat_sound, 'C:/Users/Admin/Documents/baby_filtered.wav')

print("Arquivo de som filtrado gerado: 'baby_filtered.wav'")


Arquivo de som filtrado gerado: 'baby_filtered.wav'


CODIGO FINAL

In [ ]:
import cv2
import numpy as np
import soundfile as sf
from scipy.signal import butter, lfilter

# Carregar imagem de ultrassom
def load_ultrasound(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))  
    img = img / 255.0  
    return img

# Gerar som de batidas cardíacas simulando frequência natural
def ultrasound_to_heartbeat(image):
    heartbeat = []
    for row in image:
        intensity = np.mean(row)  
        
        # Ajustar frequências para faixa cardíaca natural: 110-160 BPM
        # Frequência ajustada para batimentos de um embrião de 8 semanas
        frequency = np.interp(intensity, [0, 1], [110, 167])
        
        duration = 0.5 * (60 / frequency)  # Duração ajustada com base na frequência (BPM)
        t = np.linspace(0, duration, int(44100 * duration), endpoint=False) 
        beat = 0.5 * np.sin(2 * np.pi * frequency / 60 * t)  # Frequência convertida de BPM para Hz
        heartbeat.extend(beat)
    return np.array(heartbeat)

# Adicionar pausas para criar batimentos realistas
def add_pauses(sound, pause_duration=0.4, fs=44100):
    pause = np.zeros(int(fs * pause_duration))  
    return np.concatenate([sound, pause])

# Filtro passa-baixa para eliminar ruídos
def lowpass_filter(data, cutoff=120, fs=44100, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    return y

# Salvar som em arquivo WAV
def save_heartbeat(sound, file_path):
    sound = sound / np.max(np.abs(sound))  # Normalizar para [-1, 1]
    sf.write(file_path, sound, samplerate=44100)

# Caminho da imagem
image_path = 'frequencia_bebe.jpg'

# Processar imagem
image = load_ultrasound(image_path)

# Gerar som cardíaco
heartbeat_sound = ultrasound_to_heartbeat(image)

# Adicionar pausas entre batimentos
heartbeat_with_pauses = []
fs = 44100
for i in range(0, len(heartbeat_sound), int(fs * 0.5)):  # Intervalo ajustado para batidas naturais
    heartbeat_with_pauses.extend(add_pauses(heartbeat_sound[i:i + int(fs * 0.5)], pause_duration=0.4))
heartbeat_sound = np.array(heartbeat_with_pauses)

# Aplicar filtro para suavizar som
heartbeat_sound = lowpass_filter(heartbeat_sound)

# Salvar som final
save_heartbeat(heartbeat_sound, 'C:/Users/Admin/Documents/baby_natural.wav')

print("Arquivo com frequência natural gerado: 'baby_natural.wav'")

Arquivo com frequência natural gerado: 'baby_natural.wav'
